# Web sracper for Immoscout.ch

In [ ]:
from bs4 import BeautifulSoup
from requests import get
from lxml import etree

import csv
import time
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
headers = headers = ({'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'})

In [ ]:
base_url = "https://www.immoscout24.ch/de/immobilien/mieten/kanton-{}"
cantons = ["zuerich", "bern", "luzern", "uri", "schwyz", "obwalden", "nidwalden", "glarus", "zug", "freiburg", 
           "solothurn", "basel-stadt", "basel-landschaft", "schaffhausen", "appenzell-ai", 
           "appenzell-ar", "st-gallen", "graubuenden", "aargau", "thurgau", "tessin", "waadt", "wallis", 
           "neuenburg", "genf", "jura"]

In [ ]:
# Define the base URL and page range
base_url = "https://www.immoscout24.ch/de/immobilien/mieten/kanton-{}"

for canton in cantons:
    # Create an empty list to store the results
    results = []
    print(str(canton))
    
    canton_url = base_url.format(canton)
    response = get(canton_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
       
    pages = range(1, 43)
    
    # Loop through each page and scrape the data
    for page in pages:
        url = f"{canton_url}?pn={page}"
        print(str(page))
        response = get(url,headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        entries = soup.find_all("div", {"class": "Content-sVEls kfPvCj"})
        for entry in entries:
            header_elem = entry.find("h3", {"class": "Box-cYFBPY hKJGPR Heading-daBLVV dOtgYu"})
            if header_elem is not None:
                header = header_elem.text.strip()
                header = header.replace(' Zimmer', '').replace(' m²', '').replace(' (m², Jahr)', '').replace(' CHF','').replace('CHF','').replace('.—','').replace('EUR ', '').replace('EUR', '').replace('Preis auf Anfrage','').replace('(Monat)','').replace('(Jahr)','').replace('(Woche)','').replace('(Tag)','')
            else:
                header = ''

            address_elem = entry.find("span", {"class": "AddressLine__TextStyled-eaUAMD iBNjyG"})
            if address_elem is not None:
                address = address_elem.text.strip()
            else:
                address = ''

            title_elem = entry.find("h2", {"class": "Box-cYFBPY gbsxmK Title__TitleStyled-JDiVe zBlIG"})
            if title_elem is not None:
                title = title_elem.text.strip()
            else:
                title = ''

            desc_elem = entry.find("p", {"class": "Box-cYFBPY wYYQQ"})
            if desc_elem is not None:
                description = desc_elem.text.strip()
            else:
                description = ''

            result = {"header": header, "address": address, "title": title, "description": description}
            results.append(result)
            #time.sleep(random.uniform(1, 3))
            
    print(len(results))
    filename = f"{canton}.csv"
    df = pd.DataFrame(results)
    # Save the dataframe as a CSV file
    df.to_csv(filename, index=False)